In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"qasmovdyant","key":"4d4759066dca189587bcac8981b34740"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rohit265/loan-approval-dataset

Dataset URL: https://www.kaggle.com/datasets/rohit265/loan-approval-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 46% 5.00M/11.0M [00:00<00:00, 52.3MB/s]
100% 11.0M/11.0M [00:00<00:00, 65.9MB/s]


In [ ]:
!unzip /content/loan-approval-dataset.zip

Archive:  /content/loan-approval-dataset.zip
  inflating: loan_approval_dataset.json  


In [ ]:
df = pd.read_json('/content/loan_approval_dataset.json')

In [ ]:
df

,Income,Experience,Married/Single,House_Ownership,Car_Ownership,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1303834,3,0.0,0.0,1.0,3,13,0
1,7574516,10,0.0,0.0,1.0,9,13,0
2,3991815,4,1.0,0.0,1.0,4,10,0
3,6256451,2,0.0,0.0,0.0,2,12,1
4,5768871,11,0.0,0.0,1.0,3,14,1
...,...,...,...,...,...,...,...,...
251995,8154883,13,0.0,0.0,1.0,6,11,0
251996,2843572,10,0.0,0.0,1.0,6,11,0
251997,4522448,7,0.0,0.0,1.0,7,12,0
251998,6507128,0,0.0,0.0,1.0,0,10,0


In [ ]:
df.drop(['Id', 'Profession', 'CITY', 'STATE', 'Age'], axis = 1, inplace = True)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

In [ ]:
enc = OrdinalEncoder(categories=[['single', 'married']])
df['Married/Single'] = enc.fit_transform(df[['Married/Single']])

In [ ]:
enc = OrdinalEncoder(categories=[['rented', 'owned', 'norent_noown']])
df['House_Ownership'] = enc.fit_transform(df[['House_Ownership']])

In [ ]:
enc = OrdinalEncoder(categories=[['yes', 'no']])
df['Car_Ownership'] = enc.fit_transform(df[['Car_Ownership']])

In [ ]:
X, y = df.drop('Risk_Flag', axis = 1), df['Risk_Flag'].copy()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
num_features = X_train.select_dtypes(include = [np.number]).columns
cat_features = X_train.select_dtypes(exclude = [np.number]).columns

In [ ]:
from sklearn.pipeline import Pipeline,  make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
num_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy = 'median')),
    ('scale', StandardScaler())
])


cat_pipeline =Pipeline([
    ('impute', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
])

transformer = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features),
], remainder = 'passthrough')

estimator = LogisticRegression(class_weight = 'balanced', random_state = 42, penalty='l2')
full_pipeline = Pipeline([
    ('preprocessing', transformer),
    ('estimator', estimator)
])

In [ ]:
X_train = X_train.sample(1000, random_state=42)
y_train = y_train.sample(1000, random_state = 42)

In [ ]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  Index(['Income', 'Experience', 'Married/Single', 'House_Ownership',
       'Car_Ownership', 'CURRENT_JOB_YRS', 'CURRENT_HOUSE_YRS'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index([], dtype='object'))])),
                ('estimator',
                 LogisticRegression(class_weight='balanced', random_state=42))])

In [ ]:
print(full_pipeline.score(X_train, y_train))
print(full_pipeline.score(X_test, y_test))

0.457
0.4601190476190476


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16,
n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

print(rnd_clf.score(X_train, y_train))
print(rnd_clf.score(X_test, y_test))

0.883
0.8760119047619047


In [ ]:
class_weights = {0:0.123, 1:0.877}

log_reg = LogisticRegression(class_weight=class_weights, random_state=42)
log_reg.fit(X_train, y_train)

print(log_reg.score(X_train, y_train))
print(log_reg.score(X_test, y_test))

0.882
0.8759325396825397


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.datasets import make_regression

lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

print(lasso.coef_)
print(lasso.score(X_test, y_test))


[ 4.48678488e-10 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00]
-0.0004051114084835383


In [ ]:
accuracy = accuracy_score(y_test, y_test)
accuracy

1.0

In [ ]:
recall = recall_score(y_test, y_test)
recall

1.0

In [ ]:
f1 = f1_score(y_test, y_pred)
f1

0.0

In [ ]:
precision = precision_score(y_test, y_test)
precision

1.0

In [ ]:
roc_auc_score = roc_auc_score(y_test, y_test)
roc_auc_score

TypeError: 'numpy.float64' object is not callable